In [239]:
from bs4 import BeautifulSoup
import requests
import pickle
import json
import re

from pprint import pprint

In [199]:
url = 'https://www.cgtrader.com/3d-models/architectural/decoration/ceiling-wood-thin-branch-beam-n1'

response = requests.get(url, 'html.parser')

soup = BeautifulSoup(response.content, 'html.parser')

In [307]:
def get_tags(soup):
    tags = list(map(lambda x: x.text, soup.select('li.label')))
    return tags


def get_views(soup):
    return int(soup.select('.stats-info__views-button')[0].attrs.get('content').split(' ')[0])


def get_likes(soup):
    return int(soup.select('.stats-info__like-button > div:nth-child(1)')[0].attrs.get('content').split(' ')[0])


def get_price(soup):
    return soup.select('.product-pricing__price > span:nth-child(1) > span:nth-child(1)')[0].text


def get_pic_count(soup):
    return int(soup.select('.footerCount')[0].text.split('/')[1].strip())


def get_comments(soup):
    comments = soup.select('.comments-list')[0]
    comment_authors = list(map(lambda x: x.text, comments.select('.author-link')))
    comment_texts = list(map(lambda x: x.text[:-5], comments.select('.card')))
    comment_datetimes = list(map(lambda x: datetime.strptime(x.text, '%Y-%m-%d %H:%M'), comments.select('.project-comment__date')))
    comments_dict = {
        'authors': comment_authors, 
        'texts': comment_texts, 
        'datetimes': comment_datetimes
    }
    return comments_dict


def get_modeler(soup):
    return soup.select('.username')[0].text


def get_modeler_ratings(soup):
    ratings = {
        'avg_rating': json.loads(soup.select('div.author-rating > div:nth-child(1)')[0].attrs.get('data-react-props')).get('rating'),
        'num_rating': int(soup.select('.link--primary')[0].text[1:-1].split(' ')[0])
    }
    return ratings


def get_modeler_response(soup):
    response_arr = soup.select('.author-response > span:nth-child(2)')[0].text.split(' ')
    response = {
        'percent': int(response_arr[0][:-1]),
        'time': float(response_arr[2][:-1])
    }
    return response


def get_model_description(soup):
    description = soup.select('.product-description > p:nth-child(1)')[0].text.replace('\n', ' ')
    return description
    

def get_model_formats(soup):
    format_body = soup.select('ul.info-list')[0]
    formats = [x.text for x in format_body.select('.js-format-link')]
    sizes = [x.text for x in format_body.select('.right-column')]
    format_dict = {
        'formats': formats,
        'sizes': sizes
    }
    return format_dict

In [394]:
def is_svg_checked(tag):
    return 'is-green' in str(tag)

In [410]:
def get_model_details(soup):
    info_list = soup.select('ul.info-list:nth-child(2)')[0]
    first_two = [x for x in info_list.select('li')][0]
    publish_and_model_id_lst = list(first_two.children)
    publish_date = datetime.strptime(publish_and_model_id_lst[1].text, '%Y-%m-%d')
    model_id = publish_and_model_id_lst[2].text.split('#')[1]
    bool_cols = ['animated', 'rigged', 'vr/ar/low-poly', 'pbr', 'textures', 'materials', 'uv_mapping', 'plugins_used']
    bool_vals = list(map(lambda x: 'is-green' in str(x), info_list.select('.fal')))
    bool_dict = {k: v for k, v in list(zip(bool_cols, bool_vals))}
    geometry = info_list.select('.u-float-right')[4].text
    polygon_text = info_list.select('.u-float-right')[5].text
    polygons = int(''.join(polygon_text.split(',')))
    vertice_text = info_list.select('.u-float-right')[6].text
    vertices = int(''.join(vertice_text.split(',')))
    unwrapped_uvs = info_list.select('.u-float-right')[10].text
    model_detail_dict = {
        'model_id':model_id,
        'geometry':geometry,
        'polygons':polygons,
        'vertices':vertices,
        'unwrapped_uvs':unwrapped_uvs
    }
    model_detail_dict.update(bool_dict)
    return model_detail_dict
    
    
    
    

In [411]:
get_model_details(soup)

{'model_id': '2383583',
 'geometry': 'Polygon mesh',
 'polygons': 155000,
 'vertices': 170000,
 'unwrapped_uvs': 'Non-overlapping',
 'animated': False,
 'rigged': False,
 'vr/ar/low-poly': False,
 'pbr': False,
 'textures': True,
 'materials': True,
 'uv_mapping': True,
 'plugins_used': True}

In [388]:
geometry = info_list.select('.u-float-right')[4].text


In [360]:
info_list = soup.select('ul.info-list:nth-child(2)')[0]

In [399]:
is_svg_checked(info_list.select('.u-float-right')[7])

True

In [390]:
info_list.select('.u-float-right')

[<span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span class="right-column">Polygon mesh</span></span>,
 <span class="u-float-right"><span class="right-column">155,000</span></span>,
 <span class="u-float-right"><span class="right-column">170,000</span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-check-circle is-green is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-check-circle i

In [404]:
info_list.select('.u-float-right')[10].text

'Non-overlapping'

In [384]:
int(''.join(geometry.split(',')))

155000

In [381]:
'155555'.split(',')

['155555']

In [376]:
bool_vals = info_list.select('.u-float-right')[0:4]
bool_vals.extend(info_list.select('.u-float-right'))

[<span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span>,
 <span class="u-float-right"><span class="right-column">Polygon mesh</span></span>]

In [332]:
publish_and_model_id_lst = list(first_two.children)

In [327]:
publish_and_model_id_lst[0], datetime.strptime(publish_and_model_id_lst[1].text, '%Y-%m-%d')
publish_and_model_id_lst[2].text.split('#')

('Publish date', datetime.datetime(2020, 4, 23, 0, 0))

In [308]:
get_model_formats(soup)

{'formats': ['Autodesk 3ds Max',
  'Autodesk FBX',
  'OBJ',
  '3D Studio',
  'Stereolithography'],
 'sizes': ['-', '45.8 MB', '45.8 MB', '45.8 MB', '-']}

In [294]:
test = soup.select('ul.info-list')[0]

In [298]:
[x.text for x in test.select('.right-column')]

['-', '45.8 MB', '45.8 MB', '45.8 MB', '-']

In [293]:
test.select('.right-column right-column--narrow')

[]

In [309]:
ul.info-list:nth-child(2)

[<li><span class="js-format-link">Autodesk 3ds Max</span> (.max) (2 files)<span class="right-column right-column--narrow">-</span><span class="product-formats__version">Version:  2014</span><span class="product-formats__renderer">Renderer: V-Ray 3.2</span><span class="product-formats__version">Version:  2011</span><span class="product-formats__renderer">Renderer: V-Ray 3.2</span></li>,
 <li><span class="js-format-link">Autodesk FBX</span> 2014 (.fbx)<span class="right-column right-column--narrow">45.8 MB</span></li>,
 <li><span class="js-format-link">OBJ</span> 2014 (.obj, .mtl)<span class="right-column right-column--narrow">45.8 MB</span></li>,
 <li><span class="js-format-link">3D Studio</span> 2014 (.3ds)<span class="right-column right-column--narrow">45.8 MB</span></li>,
 <li><span class="js-format-link">Stereolithography</span>  (.stl) <span class="right-column">-</span><span class="js-track-link-near-formats-click js-auth-control"><span data-react-cache-id="Items/ProductActionButt

In [273]:
map(lambda x: (x.select('.js-format-link')[0].text,
               x.select('get_model_formats(soup)[0]

<span class="js-format-link">3D Studio</span>

In [117]:
list(map(lambda x: x.text, get_model_details(soup)[0]))

['Autodesk 3ds Max (.max) (2 files)-Version:  2016Renderer: V-Ray Version:  2013Renderer: V-Ray ']

In [124]:
model_detail_section = soup.select('ul.info-list:nth-child(2)')[0]

In [135]:
print(model_detail_section.select('.js-format-link')[0].text)
print(model_detail_section.select('.product-formats__version')[0].text)
print(model_detail_section.select('.product-formats__renderer')[0].text)

Autodesk 3ds Max
Version:  2016
Renderer: V-Ray 


In [144]:
ul = soup.select('ul.info-list:nth-child(2)')[1]

In [158]:
li=soup.select('ul.info-list:nth-child(2) > li:nth-child(1)')[-1]

In [168]:
get_model_details(soup)[1]

<ul class="info-list"><li>Publish date<span class="right-column">2020-03-18</span><li>Model ID<span class="right-column">#2320671</span></li></li><li>Animated <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span><div class="approve-buttons is-hidden js-auto-approve"><input class="js-approve radio" data-item="2320671" id="button-approve-animated" name="animated" type="radio" value="true"/><label class="btn approve-button has-tooltip" for="button-approve-animated" title="Select to approve">Approved</label><input checked="checked" class="js-disapprove radio" data-item="2320671" id="button-disapprove-animated" name="animated" type="radio" value="false"/><label class="btn approve-button has-tooltip" for="button-disapprove-animated" title="Select to disapprove">Disapproved</label></div></li><li>Rigged <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column">

In [178]:
soup.select('ul.info-list:nth-child(2) > li:nth-child(3) > span:nth-child(1) > span:nth-child(1) > i:nth-child(1)')[0]

<i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i>

In [194]:
soup.select('ul.info-list')[2].find_all('li')

[<li>Publish date<span class="right-column">2020-03-18</span><li>Model ID<span class="right-column">#2320671</span></li></li>,
 <li>Model ID<span class="right-column">#2320671</span></li>,
 <li>Animated <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span><div class="approve-buttons is-hidden js-auto-approve"><input class="js-approve radio" data-item="2320671" id="button-approve-animated" name="animated" type="radio" value="true"/><label class="btn approve-button has-tooltip" for="button-approve-animated" title="Select to approve">Approved</label><input checked="checked" class="js-disapprove radio" data-item="2320671" id="button-disapprove-animated" name="animated" type="radio" value="false"/><label class="btn approve-button has-tooltip" for="button-disapprove-animated" title="Select to disapprove">Disapproved</label></div></li>,
 <li>Rigged <span class="u-float-right"><span><i class="fal svg-inline--fa fa-t

In [188]:
def 'is-red' in soup.select('ul.info-list:nth-child(2) > li:nth-child(3) > span:nth-child(1) > span:nth-child(1) > i:nth-child(1)')[0].attrs.get('class')

True

In [173]:
soup.select('ul.info-list:nth-child(2) > li:nth-child(10) > span:nth-child(1) > span:nth-child(1) > i:nth-child(1)')

[<i class="fal svg-inline--fa fa-check-circle is-green is-not-spaced right-column"></i>]

In [304]:
model_detail_section = soup.select('ul.info-list:nth-child(2)')[0]

In [306]:
model_detail_section

<ul class="info-list"><li>Publish date<span class="right-column">2020-04-23</span><li>Model ID<span class="right-column">#2383583</span></li></li><li>Animated <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column"></i></span></span><div class="approve-buttons is-hidden js-auto-approve"><input class="js-approve radio" data-item="2383583" id="button-approve-animated" name="animated" type="radio" value="true"/><label class="btn approve-button has-tooltip" for="button-approve-animated" title="Select to approve">Approved</label><input checked="checked" class="js-disapprove radio" data-item="2383583" id="button-disapprove-animated" name="animated" type="radio" value="false"/><label class="btn approve-button has-tooltip" for="button-disapprove-animated" title="Select to disapprove">Disapproved</label></div></li><li>Rigged <span class="u-float-right"><span><i class="fal svg-inline--fa fa-times-circle is-red is-not-spaced right-column">

In [305]:
model_detail_section.select('.js-format-link')

[]

In [301]:
[x.text for x in model_detail_section[0][1].select('.right-column')]

AttributeError: ResultSet object has no attribute 'select'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [169]:
[x.text for x in get_model_details(soup)[1].select('.right-column')]

['2020-03-18',
 '#2320671',
 '',
 '',
 '',
 '',
 'Polygon mesh',
 '185,576',
 '286.540',
 '',
 '',
 '',
 'Unknown',
 '']

In [165]:
list(map(lambda x: x.text, get_model_details(soup)[1]))

['Publish date2020-03-18Model ID#2320671',
 'Animated ApprovedDisapproved',
 'Rigged ApprovedDisapproved',
 'VR / AR / Low-poly ApprovedDisapproved',
 'PBR ApprovedDisapproved',
 'Geometry Polygon mesh',
 'Polygons 185,576',
 'Vertices 286.540',
 'Textures ',
 'Materials ',
 'UV Mapping ',
 'Unwrapped UVs Unknown',
 'Plugins used ']

In [299]:
pprint({
    'modeler':get_modeler(soup),
    'tags':get_tags(soup),
    'views':get_likes(soup),
    'comments':get_comments(soup),
    'price':get_price(soup),
    'pic_count':get_pic_count(soup)
})

{'comments': {'authors': ['renderee',
                          'kmutlu',
                          'ViTrend',
                          'bbaltatsar',
                          'jd94'],
              'datetimes': [datetime.datetime(2020, 4, 23, 8, 32),
                            datetime.datetime(2020, 4, 23, 18, 12),
                            datetime.datetime(2020, 6, 23, 8, 47),
                            datetime.datetime(2020, 6, 26, 16, 54),
                            datetime.datetime(2020, 6, 27, 10, 44)],
              'texts': ['good job', 'nice', 'pretty', 'nice', 'awesome work!']},
 'modeler': '3d-vlad',
 'pic_count': 19,
 'price': '$6.00',
 'tags': ['ceiling',
          'bamboo',
          'branch',
          'tree',
          'decor',
          'wall',
          'panel',
          'binder',
          'cane',
          'post',
          'plant',
          'stalk',
          'restaurant',
          'bough',
          'wickerwork',
          'sang',
          'wood',
  